In [2]:
from ActiveCritic.model_src.transformer import ModelSetup, TransformerModel, CriticTransformer
import torch as th
from ActiveCritic.utils.pytorch_utils import calcMSE

In [3]:
ms = ModelSetup()
seq_len = 6
ntoken = 1
batch_size = 4
d_output = 1
ms.d_output = d_output
ms.nhead = 1
ms.d_hid = 10
ms.d_model = 10
ms.nlayers = 2
ms.seq_len = seq_len
ms.dropout = 0
ms.ntoken = 1
ms.lr = None
ms.device = 'cuda'
ms.optimizer_class = th.optim.AdamW
ms.optimizer_kwargs = {}
ms.model_class:TransformerModel = TransformerModel

In [10]:
from ActiveCritic.tests.test_utils.utils import make_seq_encoding_data, make_mask_data
inpt_seq, outpt_seq = make_seq_encoding_data(batch_size, seq_len, ntoken)


In [5]:
model = TransformerModel(model_setup=ms).to('cuda')
with th.no_grad():
    model.forward(inpt_seq)
optimizer = th.optim.Adam(params=model.parameters(), lr=1e-3)
loss = 0
for i in range(1000):
    result = model.forward(inpt_seq)
    loss = calcMSE(result, outpt_seq)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [7]:
from ActiveCritic.model_src.transformer import generate_square_subsequent_mask

In [23]:
mask = generate_square_subsequent_mask(seq_len).to('cuda')

In [30]:
inpt_seq = th.ones([2,seq_len,1], dtype=th.float, device='cuda')
inpt_seq[0,-1,0] = 0

outpt_seq = th.ones_like(inpt_seq)
outpt_seq[0] = 0

In [11]:
inpt_seq, outpt_seq, mask = make_mask_data(batch_size=batch_size, seq_len=seq_len, ntoken=ntoken)


In [16]:
model = TransformerModel(model_setup=ms).to('cuda')
with th.no_grad():
    model.forward(inpt_seq)
optimizer = th.optim.Adam(params=model.parameters(), lr=1e-3)
loss = 0
for i in range(1000):
    result = model.forward(inpt_seq, mask=None)
    loss = calcMSE(result, outpt_seq)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [17]:
loss

tensor(8.9355e-05, device='cuda:0', grad_fn=<MeanBackward0>)

In [18]:
from ActiveCritic.model_src.transformer import CriticTransformer

In [19]:
ms = ModelSetup()
seq_len = 6
d_output = 1
ms.d_output = d_output
ms.nhead = 1
ms.d_hid = 10
ms.d_model = 10
ms.nlayers = 2
ms.seq_len = seq_len
ms.dropout = 0
ms.ntoken = 1
ms.lr = None
ms.device = 'cuda'
ms.optimizer_class = th.optim.AdamW
ms.optimizer_kwargs = {}
ms.d_result = 1
ms.model_class:TransformerModel = CriticTransformer

In [27]:
inpt_seq = th.ones([2,seq_len,4], dtype=th.float, device='cuda')
inpt_seq[0,-1,0] = 0

outpt_seq = th.ones([2,1], dtype=th.float, device='cuda')
outpt_seq[0] = 0

In [28]:
model = CriticTransformer(model_setup=ms).to('cuda')
with th.no_grad():
    a = model.forward(inpt_seq)

In [29]:
a.shape

torch.Size([2, 1])

In [31]:
model = CriticTransformer(model_setup=ms).to('cuda')
with th.no_grad():
    model.forward(inpt_seq)
optimizer = th.optim.Adam(params=model.parameters(), lr=1e-3)
loss = 0
for i in range(2000):
    result = model.forward(inpt_seq)
    loss = calcMSE(result, outpt_seq)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [32]:
loss

tensor(0., device='cuda:0', grad_fn=<MeanBackward0>)